In [47]:
import glob
import pandas as pd

### Read in all the datas

In [57]:
import csv
datas = []
for file in glob.glob("well productions/*"):
    if "well production.csv" not in file:
        frame = pd.read_csv(file)
        frame["Name"] = file
        datas.append(frame)

In [58]:
datas[-1]

,easting,northing,porosity,permeability,Poisson's ratio,Young's Modulus,water saturation,oil saturation,proppant weight (lbs),pump rate (cubic feet/min),Name
0,51300.0,85900.0,0.03,0.004,0.297,8645249.897,0.102911,0.897089,597274.448817,285.239279,well productions/Brisket 7-212H.csv
1,51351.0,85900.0,0.06,0.014,0.297,8657127.872,0.103389,0.896611,NaN,NaN,well productions/Brisket 7-212H.csv
2,51402.0,85900.0,0.06,0.014,0.297,8657127.872,0.103389,0.896611,NaN,NaN,well productions/Brisket 7-212H.csv
3,51453.0,85900.0,0.06,0.015,0.297,8669066.730,0.103870,0.896130,542560.889895,207.558468,well productions/Brisket 7-212H.csv
4,51503.0,85900.0,0.06,0.015,0.297,8669066.730,0.103870,0.896130,NaN,NaN,well productions/Brisket 7-212H.csv
...,...,...,...,...,...,...,...,...,...,...,...
95,56132.0,85900.0,0.03,0.004,0.305,9293349.678,0.128985,0.871015,NaN,NaN,well productions/Brisket 7-212H.csv
96,56183.0,85900.0,0.07,0.020,0.305,9308610.864,0.129581,0.870419,544484.062085,342.719139,well productions/Brisket 7-212H.csv
97,56234.0,85900.0,0.07,0.020,0.305,9308610.864,0.129581,0.870419,NaN,NaN,well productions/Brisket 7-212H.csv
98,56284.0,85900.0,0.04,0.006,0.305,9323938.224,0.130178,0.869822,NaN,NaN,well productions/Brisket 7-212H.csv


In [71]:
def well_length(dataframe: pd.DataFrame):
    dataframe["well length"] = dataframe["easting"].iloc[-1] - dataframe["easting"][0]

In [104]:
def frac_stages(dataframe : pd.DataFrame):
    dataframe["frac stages"] = dataframe[dataframe["proppant weight (lbs)"].isna() == False].shape[0]

In [105]:
def propant_per_stage(dataframe: pd.DataFrame):
    pass

In [106]:
%%capture
list(map(well_length, datas))
list(map(frac_stages, datas))

In [108]:
datas[1].head()

,easting,northing,porosity,permeability,Poisson's ratio,Young's Modulus,water saturation,oil saturation,proppant weight (lbs),pump rate (cubic feet/min),Name,well length,frac stages
0,84500.0,37600.0,0.06,0.015,0.34,1.097546e+07,0.286440,0.713560,406263.949202,229.721504,well productions/Burger 6-148H.csv,4369.0,25
1,84544.0,37600.0,0.06,0.015,0.34,1.097546e+07,0.286440,0.713560,NaN,NaN,well productions/Burger 6-148H.csv,4369.0,25
2,84588.0,37600.0,0.07,0.019,0.34,1.097392e+07,0.287223,0.712777,NaN,NaN,well productions/Burger 6-148H.csv,4369.0,25
3,84632.0,37600.0,0.07,0.019,0.34,1.097392e+07,0.287223,0.712777,NaN,NaN,well productions/Burger 6-148H.csv,4369.0,25
4,84677.0,37600.0,0.06,0.014,0.34,1.097232e+07,0.288002,0.711998,414590.089822,309.953442,well productions/Burger 6-148H.csv,4369.0,25


In [111]:
datas[1]

,easting,northing,porosity,permeability,Poisson's ratio,Young's Modulus,water saturation,oil saturation,proppant weight (lbs),pump rate (cubic feet/min),Name,well length,frac stages
0,84500.0,37600.0,0.06,0.015,0.340,1.097546e+07,0.286440,0.713560,406263.949202,229.721504,well productions/Burger 6-148H.csv,4369.0,25
1,84544.0,37600.0,0.06,0.015,0.340,1.097546e+07,0.286440,0.713560,NaN,NaN,well productions/Burger 6-148H.csv,4369.0,25
2,84588.0,37600.0,0.07,0.019,0.340,1.097392e+07,0.287223,0.712777,NaN,NaN,well productions/Burger 6-148H.csv,4369.0,25
3,84632.0,37600.0,0.07,0.019,0.340,1.097392e+07,0.287223,0.712777,NaN,NaN,well productions/Burger 6-148H.csv,4369.0,25
4,84677.0,37600.0,0.06,0.014,0.340,1.097232e+07,0.288002,0.711998,414590.089822,309.953442,well productions/Burger 6-148H.csv,4369.0,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,88693.0,37600.0,0.07,0.020,0.341,1.085080e+07,0.315022,0.684978,NaN,NaN,well productions/Burger 6-148H.csv,4369.0,25
96,88737.0,37600.0,0.07,0.020,0.341,1.085080e+07,0.315022,0.684978,583013.844447,252.755942,well productions/Burger 6-148H.csv,4369.0,25
97,88781.0,37600.0,0.04,0.005,0.341,1.084608e+07,0.315582,0.684418,NaN,NaN,well productions/Burger 6-148H.csv,4369.0,25
98,88825.0,37600.0,0.04,0.005,0.341,1.084608e+07,0.315582,0.684418,NaN,NaN,well productions/Burger 6-148H.csv,4369.0,25
